In [115]:
'''
This notebook is oriented to use a given dataset customized to contains 6 features and training a classification tree
This features are: cqt, rmse, energy, mfccs, chromagram and spectral contrast
This notebook is currently in development phase, so contains a considerably quantity of test code

'''

'\nThis notebook is oriented to use a given dataset customized to contains 6 features and training a classification tree\nThis features are: cqt, rmse, energy, mfccs, chromagram and spectral contrast\nThis notebook is currently in development phase, so contains a considerably quantity of test code\n\n'

In [11]:
import tensorflow as tf
import pandas as pd
from pathlib import Path
import librosa, librosa.display
import matplotlib.pyplot as plt
import numpy as np
import gc

# Reading Audio Data


# features = pd.read_csv('Datasets/2000Spoti.csv', encoding='latin-1', thousands=',')

features = pd.read_csv('Dataframes/var_mean_dict.csv', encoding='latin-1', thousands=',') # Media genera 0.62-0.67 con todas las caracteristicas **best result2**

print(features.head(5))

print('done')

   Unnamed: 0  Genre  Energy_Mean  RMSE_Mean  ZCR_Mean  CQT_Mean  \
0           0  Blues    40.490492   0.130192     55050  0.001573   
1           1  Blues    23.686239   0.095925     37156  0.003940   
2           2  Blues    68.631996   0.175582     50560  0.007814   
3           3  Blues    53.671735   0.141053     22060  0.003624   
4           4  Blues    21.844622   0.091499     67244  0.001955   

   Spectral_Contrast_Mean  Chromagram_Mean  Tempo_Mean  STFT_Mean  ...  \
0               20.526699         0.278385  123.046875   0.000037  ...   
1               20.676128         0.266356  107.666016   0.000021  ...   
2               22.197517         0.272697  161.499023   0.000009  ...   
3               21.426268         0.252442  172.265625   0.000005  ...   
4               21.466338         0.269720  135.999178   0.000014  ...   

   MFCCS3  MFCCS4  MFCCS5  MFCCS6  MFCCS7  MFCCS8  MFCCS9  MFCCS10  MFCCS11  \
0     226     125     114      82      44      39      31       57 

In [12]:

#extracting labels and features

labelColumn = 'Genre'

labels = np.array(features[labelColumn])
print(np.unique(labels))

print("The labels of out dataset are:", np.unique(labels))
print("And their shape is:", labels.shape)


# Only var no CQT
# featureColumsToDrop = ['Unnamed: 0','STFT_Mean.1', 'CQT_Var', 'STFT_Var.1', 'Genre', 'Energy_Mean', 'RMSE_Mean', 'ZCR_Mean', 'CQT_Mean', 'Spectral_Contrast_Mean', 'Chromagram_Mean', 'Tempo_Mean', 'STFT_Mean', 
#                       'Autocorrelation_Mean', 'STFT_Mean', 'Spectral_Centroid_Mean', 'Rolloff_Mean']

# Only var with CQT
# featureColumsToDrop = ['Unnamed: 0', 'Energy_Mean', 'CQT_Var', 'ZCR_Mean', 'CQT_Mean', 'Spectral_Contrast_Mean', 'Chromagram_Mean', 'Tempo_Mean', 'STFT_Mean', 
#                       'Autocorrelation_Mean', 'STFT_Mean', 'Spectral_Centroid_Mean', 'Rolloff_Mean','STFT_Mean.1', 'STFT_Var.1', 'Genre']

# Only mean no CQT

# featureColumsToDrop = ['Unnamed: 0','Energy_Var', 'RMSE_Var', 'ZCR_Var', 'CQT_Var', 'Spectral_Contrast_Var', 'Chromagram_Var', 'Tempo_Var', 'STFT_Var',
#                       'Autocorrelation_Var', 'STFT_Var', 'Spectral_Centroid_Var', 'Rolloff_Var', 'CQT_Mean', 'STFT_Mean.1', 'STFT_Var.1', 'Genre']

# Only mean with CQT
# featureColumsToDrop = ['Unnamed: 0', 'Energy_Var', 'RMSE_Var', 'ZCR_Var', 'CQT_Var', 'Spectral_Contrast_Var', 'Chromagram_Var', 'Tempo_Var', 'STFT_Var',
#                       'Autocorrelation_Var', 'STFT_Var', 'Spectral_Centroid_Var', 'Rolloff_Var',  'STFT_Mean.1', 'STFT_Var.1', 'Genre']


# Mean and var with CQT

# featureColumsToDrop = ['Unnamed: 0', 'STFT_Mean.1', 'STFT_Var.1', 'Genre']


# NO MFCCS mean and var

# featureColumsToDrop = ['Unnamed: 0', 'STFT_Mean.1', 'STFT_Var.1', 'Genre',
#                        'Energy_Mean', 'RMSE_Mean', 'ZCR_Mean', 'CQT_Mean', 'Spectral_Contrast_Mean', 'Chromagram_Mean', 'Tempo_Mean', 'STFT_Mean', 
#                       'Autocorrelation_Mean', 'STFT_Mean', 'Spectral_Centroid_Mean', 'Rolloff_Mean',
#                        'MFCCS0','MFCCS1','MFCCS2','MFCCS3','MFCCS4','MFCCS5','MFCCS6','MFCCS7','MFCCS8','MFCCS9','MFCCS10', 'CQT_Mean','MFCCS11','MFCCS12', 'CQT_Var', 'STFT_Var.1', 'Genre']  


# First prove that was made

featureColumsToDrop = ['Unnamed: 0', 'Autocorrelation_Mean', 'STFT_Mean', 'Rolloff_Mean', 'Energy_Var', 'RMSE_Var', 'ZCR_Var', 'CQT_Var', 'Spectral_Contrast_Var', 'Chromagram_Var', 'Tempo_Var', 'STFT_Var',
                      'Autocorrelation_Var', 'STFT_Var', 'Spectral_Centroid_Var', 'Rolloff_Var', 'CQT_Mean', 'STFT_Mean.1', 'STFT_Var.1', 'Genre']

features = features.drop(featureColumsToDrop, 1)

print("The features of out dataset are:")

features.head(5)

['Blues' 'Classical' 'Country' 'Disco' 'Hiphop' 'Jazz' 'Metal' 'Pop'
 'Reggae' 'Rock']
The labels of out dataset are: ['Blues' 'Classical' 'Country' 'Disco' 'Hiphop' 'Jazz' 'Metal' 'Pop'
 'Reggae' 'Rock']
And their shape is: (1000,)
The features of out dataset are:


,Energy_Mean,RMSE_Mean,ZCR_Mean,Spectral_Contrast_Mean,Chromagram_Mean,Tempo_Mean,Spectral_Centroid_Mean,MFCCS0,MFCCS1,MFCCS2,MFCCS3,MFCCS4,MFCCS5,MFCCS6,MFCCS7,MFCCS8,MFCCS9,MFCCS10,MFCCS11,MFCCS12
0,40.490492,0.130192,55050,20.526699,0.278385,123.046875,1784.165850,2524,331,260,226,125,114,82,44,39,31,57,60,35
1,23.686239,0.095925,37156,20.676128,0.266356,107.666016,1530.176679,8081,670,787,774,341,139,94,82,75,64,83,81,55
2,68.631996,0.175582,50560,22.197517,0.272697,161.499023,1552.811865,3447,397,491,197,252,85,80,59,34,39,40,40,51
3,53.671735,0.141053,22060,21.426268,0.252442,172.265625,1070.106615,5910,378,852,327,183,108,50,55,36,82,48,40,35
4,21.844622,0.091499,67244,21.466338,0.269720,135.999178,1835.004266,5897,372,1344,187,150,54,76,52,26,107,66,49,37


In [13]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from matplotlib.pyplot import figure
from sklearn.model_selection import train_test_split

def train_clasify():
    """
    Method that train a RF Classifier and return a metrics report with
        - Recall by label
        - Precision by label
        - F1-score by label
        - Avg weighted recall
        - Avg weighted precision
        - Avg weighted f1-score
        - Avg macro recall
        - Avg macro recall
        - Avg macro recall
        - Avg total accuracy
    """

    # Split the data into training and testing sets
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25)

    print('Training Features Shape:', train_features.shape)
    print('Training Labels Shape:', train_labels.shape)
    print('Testing Features Shape:', test_features.shape)
    print('Testing Labels Shape:', test_labels.shape)

    # Instantiate model with 1000 decision trees
    rf = RandomForestClassifier(n_estimators = 1000)

    # Train the model on training data
    rf.fit(train_features, train_labels)

    print("Features Importances")
    print(rf.feature_importances_)

    predictions = rf.predict(test_features)


    report = classification_report(test_labels, predictions, np.unique(labels))
    return report

def get_metrics(report, tag, start_point):
    """
    Method that extract metrics from the string table returned by classification_report method from sklearn package
    """
    avg_precision = report[report.index(tag)+start_point:report.index(tag)+start_point+4]
    avg_recall = report[report.index(tag)+start_point+10:report.index(tag)+start_point+10+4]
    avg_f1_score = report[report.index(tag)+start_point+20:report.index(tag)+start_point+20+4]
    return avg_precision, avg_recall, avg_f1_score

def get_metrics_avg(number_iterations):
    """
    Method that make number_iterations training a RF classifier and return 2 dictionaires with metics
    
        - One with metrics by genre label
        - The other with total metrics of all the genre labels
    """
    
    # Initializing the by genre and total metrics arrays necessaries to catch the data
    
    blues_precision_array = np.arange(number_iterations).astype(np.float32)
    blues_recall_array = np.arange(number_iterations).astype(np.float32)
    blues_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    classical_precision_array = np.arange(number_iterations).astype(np.float32)
    classical_recall_array = np.arange(number_iterations).astype(np.float32)
    classical_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    country_precision_array = np.arange(number_iterations).astype(np.float32)
    country_recall_array = np.arange(number_iterations).astype(np.float32)
    country_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    disco_precision_array = np.arange(number_iterations).astype(np.float32)
    disco_recall_array = np.arange(number_iterations).astype(np.float32)
    disco_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    hiphop_precision_array = np.arange(number_iterations).astype(np.float32)
    hiphop_recall_array = np.arange(number_iterations).astype(np.float32)
    hiphop_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    jazz_precision_array = np.arange(number_iterations).astype(np.float32)
    jazz_recall_array = np.arange(number_iterations).astype(np.float32)
    jazz_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    metal_precision_array = np.arange(number_iterations).astype(np.float32)
    metal_recall_array = np.arange(number_iterations).astype(np.float32)
    metal_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    pop_precision_array = np.arange(number_iterations).astype(np.float32)
    pop_recall_array = np.arange(number_iterations).astype(np.float32)
    pop_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    reggae_precision_array = np.arange(number_iterations).astype(np.float32)
    reggae_recall_array = np.arange(number_iterations).astype(np.float32)
    reggae_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    rock_precision_array = np.arange(number_iterations).astype(np.float32)
    rock_recall_array = np.arange(number_iterations).astype(np.float32)
    rock_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    macro_avg_precision_array = np.arange(number_iterations).astype(np.float32)
    macro_avg_recall_array = np.arange(number_iterations).astype(np.float32)
    macro_avg_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    weighted_avg_precision_array = np.arange(number_iterations).astype(np.float32)
    weighted_avg_recall_array = np.arange(number_iterations).astype(np.float32)
    weighted_avg_f1_score_array = np.arange(number_iterations).astype(np.float32)
    
    accuracy_avg_array = np.arange(number_iterations).astype(np.float32)
    
    labels = ['Blues','Classical','Country','Disco','Hiphop','Jazz','Metal','Pop','Reggae','Rock']
    
    # Loop with the training RF classiffier itrerations. We fill the arrays metrics in each iteration     
    
    for i in range(0, number_iterations):
        report = train_clasify()
        
        # Metrics by Genre
        (precision, recall, f1_score) = get_metrics(report, "Blues", 12)
        blues_precision_array[i] = precision
        blues_recall_array[i] = recall
        blues_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Classical", 16)
        classical_precision_array[i] = precision
        classical_recall_array[i] = recall
        classical_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Country", 14)
        country_precision_array[i] = precision
        country_recall_array[i] = recall
        country_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Disco", 12)
        disco_precision_array[i] = precision
        disco_recall_array[i] = recall
        disco_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Hiphop", 13)
        hiphop_precision_array[i] = precision
        hiphop_recall_array[i] = recall
        hiphop_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Jazz", 11)
        jazz_precision_array[i] = precision
        jazz_recall_array[i] = recall
        jazz_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Metal", 12)
        metal_precision_array[i] = precision
        metal_recall_array[i] = recall
        metal_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Pop", 10)
        pop_precision_array[i] = precision
        pop_recall_array[i] = recall
        pop_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Reggae", 13)
        reggae_precision_array[i] = precision
        reggae_recall_array[i] = recall
        reggae_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "Rock", 11)
        rock_precision_array[i] = precision
        rock_recall_array[i] = recall
        rock_f1_score_array[i] = f1_score
        
        # General metrics         
        
        (precision, recall, f1_score) = get_metrics(report, "accuracy", 15)
        accuracy_avg_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "weighted", 19)
        weighted_avg_precision_array[i] = precision
        weighted_avg_recall_array[i] = recall
        weighted_avg_f1_score_array[i] = f1_score
        
        (precision, recall, f1_score) = get_metrics(report, "macro avg", 16)
        macro_avg_precision_array[i] = precision
        macro_avg_recall_array[i] = recall
        macro_avg_f1_score_array[i] = f1_score
        
    # Now we fill the arrays that we are going to use to create the final dictionaries     
    
    precision_array = []
    recall_array = []
    f1_score_array = []
    
    precision_array.append(blues_precision_array.mean())
    recall_array.append(blues_recall_array.mean())
    f1_score_array.append(blues_f1_score_array.mean())
    
    precision_array.append(classical_precision_array.mean())
    recall_array.append(classical_recall_array.mean())
    f1_score_array.append(classical_f1_score_array.mean())
    
    precision_array.append(country_precision_array.mean())
    recall_array.append(country_recall_array.mean())
    f1_score_array.append(country_f1_score_array.mean())
    
    precision_array.append(disco_precision_array.mean())
    recall_array.append(disco_recall_array.mean())
    f1_score_array.append(disco_f1_score_array.mean())
    
    precision_array.append(hiphop_precision_array.mean())
    recall_array.append(hiphop_recall_array.mean())
    f1_score_array.append(hiphop_f1_score_array.mean())
    
    precision_array.append(jazz_precision_array.mean())
    recall_array.append(jazz_recall_array.mean())
    f1_score_array.append(jazz_f1_score_array.mean())
    
    precision_array.append(metal_precision_array.mean())
    recall_array.append(metal_recall_array.mean())
    f1_score_array.append(metal_f1_score_array.mean())
    
    precision_array.append(pop_precision_array.mean())
    recall_array.append(pop_recall_array.mean())
    f1_score_array.append(pop_f1_score_array.mean())
    
    precision_array.append(reggae_precision_array.mean())
    recall_array.append(reggae_recall_array.mean())
    f1_score_array.append(reggae_f1_score_array.mean())
    
    precision_array.append(rock_precision_array.mean())
    recall_array.append(rock_recall_array.mean())
    f1_score_array.append(rock_f1_score_array.mean())
    
    # Dictionaries creation     
    
    by_label_metrics_dict = {
        
        'Genre': labels,
        'avg_precision': precision_array,
        'avg_recall': recall_array,
        'avg_f1_score': f1_score_array,
        
    }
    
    total_metrics_dict = {
        
        'avg_accuracy': accuracy_avg_array.mean(),
        
        'weighted_avg_precision': weighted_avg_precision_array.mean(),
        'weighted_avg_recall': weighted_avg_recall_array.mean(),
        'weighted_avg_f1_score': weighted_avg_f1_score_array.mean(),
        
        'macro_avg_precision': macro_avg_precision_array.mean(),
        'macro_avg_recall': macro_avg_recall_array.mean(),
        'macro_avg_f1_score': macro_avg_f1_score_array.mean(),
        
    }

    return by_label_metrics_dict, total_metrics_dict


def export_genre_result_dict_to_csv(by_label_metrics_dict, total_metrics_dict):
    """
    Method that export two dictionaries to csv files using pandas dataframes
    
        - One with metrics by genre label
        - The other with total metrics of all the genre labels
    """
    by_label_metrics_dataframe_columns_tags = ['Genre', 'avg_precision', 'avg_recall', 'avg_f1_score']
    total_metrics_dataframe_columns_tags = ['avg_accuracy', 
                                            'weighted_avg_precision', 'weighted_avg_recall', 'weighted_avg_f1_score',
                                            'macro_avg_precision', 'macro_avg_recall', 'macro_avg_f1_score'
                                           ]

    df_by_label = pd.DataFrame(by_label_metrics_dict, columns = by_label_metrics_dataframe_columns_tags)
    df_total = pd.DataFrame(total_metrics_dict, columns = total_metrics_dataframe_columns_tags, index=[0])

    df_by_label.to_csv('metrics/by_label_metrics_first_prove.csv')
    df_total.to_csv('metrics/total_metrics_first_prove.csv')
    
    
(by_label_metrics_dict, total_metrics_dict) = get_metrics_avg(100)
export_genre_result_dict_to_csv(by_label_metrics_dict, total_metrics_dict)


Training Features Shape: (750, 20)
Training Labels Shape: (750,)
Testing Features Shape: (250, 20)
Testing Labels Shape: (250,)
Features Importances
[0.07026758 0.06880423 0.06845423 0.07243751 0.06002534 0.03327459
 0.0844036  0.05021744 0.04416483 0.03276663 0.04147238 0.03660785
 0.03735368 0.04618366 0.038992   0.04276944 0.04384677 0.04530622
 0.04244486 0.04020716]
Training Features Shape: (750, 20)
Training Labels Shape: (750,)
Testing Features Shape: (250, 20)
Testing Labels Shape: (250,)


C:\Users\alvar\anaconda3\envs\python37\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['Blues' 'Classical' 'Country' 'Disco' 'Hiphop' 'Jazz' 'Metal' 'Pop'
 'Reggae' 'Rock'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Features Importances
[0.07775207 0.07412395 0.06578988 0.07340566 0.05482348 0.02982598
 0.07914209 0.05140985 0.04362068 0.03333578 0.04325811 0.03579028
 0.03685841 0.04277814 0.03761179 0.04132874 0.04535562 0.04430233
 0.04632263 0.04316454]
Training Features Shape: (750, 20)
Training Labels Shape: (750,)
Testing Features Shape: (250, 20)
Testing Labels Shape: (250,)


C:\Users\alvar\anaconda3\envs\python37\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['Blues' 'Classical' 'Country' 'Disco' 'Hiphop' 'Jazz' 'Metal' 'Pop'
 'Reggae' 'Rock'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Features Importances
[0.0728232  0.06973357 0.0636381  0.07079769 0.05746299 0.03361969
 0.08374497 0.05511369 0.04603271 0.03432642 0.04482731 0.03732477
 0.03476016 0.04438808 0.03803899 0.04161934 0.04581867 0.0439903
 0.04589097 0.03604837]
Training Features Shape: (750, 20)
Training Labels Shape: (750,)
Testing Features Shape: (250, 20)
Testing Labels Shape: (250,)


...

C:\Users\alvar\anaconda3\envs\python37\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['Blues' 'Classical' 'Country' 'Disco' 'Hiphop' 'Jazz' 'Metal' 'Pop'
 'Reggae' 'Rock'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Features Importances
[0.07563141 0.07448265 0.06391054 0.07516615 0.06156708 0.03229078
 0.08065365 0.0514748  0.0449215  0.03214699 0.04061835 0.03743526
 0.03350226 0.04192878 0.04054136 0.04037407 0.04389972 0.04355056
 0.04404366 0.04186044]


C:\Users\alvar\anaconda3\envs\python37\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['Blues' 'Classical' 'Country' 'Disco' 'Hiphop' 'Jazz' 'Metal' 'Pop'
 'Reggae' 'Rock'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Training Features Shape: (750, 20)
Training Labels Shape: (750,)
Testing Features Shape: (250, 20)
Testing Labels Shape: (250,)
Features Importances
[0.07332436 0.07446343 0.0634789  0.06930711 0.05646846 0.03475229
 0.08563869 0.05063778 0.04534717 0.03363194 0.04052832 0.03829653
 0.035355   0.04628562 0.03907992 0.04019496 0.04285643 0.04289145
 0.04352079 0.04394086]


C:\Users\alvar\anaconda3\envs\python37\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['Blues' 'Classical' 'Country' 'Disco' 'Hiphop' 'Jazz' 'Metal' 'Pop'
 'Reggae' 'Rock'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [14]:
import junix;
junix.export_images(filepath= "ClassificationTree.ipynb")


In [15]:
plot_confusion_matrix(rf, test_features, test_labels)
fig = plt.gcf()
fig.set_size_inches(28.5, 28.5)
plt.show()

import junix;
junix.export_images(filepath= "ClassificationTree.ipynb")

NameError: name 'rf' is not defined